# Cálculo de pontos fixos e sua estabilidade

Este é um guia para calcular os pontos fixos de um modelo, e posteriormente suas condições para estabilidade, utilizando a biblioteca *Sympy*.

### Por que usar Sympy?

O *Sympy* é um pacote que trabalha com linguagem matemática simbólica. Ou seja, o foco dos cálculos são símbolos e não números. Com o *Sympy* é possível trabalhar com expressões matemátical sem precisar designar valore às variáveis e parâmetros. Com esta ferramenta, podemos calcular expressões analíticas dos pontos fixos de um modelo, assim como sua matriz jacobiana 

Começamos o algoritmo  importando a biblioteca *Sympy*:

In [ ]:
from sympy import *

## Definindo um modelo

O próximo passo é definir o modelo a ser analisado. Nesse guia, escolhemos um modelo de duas espécies que competem pelo mesmo recurso, dado por:

$$\displaystyle \dfrac{d u_1}{dt} = r_1 u_1 \left(1- \dfrac{u_1}{K_1} - b_{12}\dfrac{u_2}{K_2}\right)$$
$$\displaystyle \dfrac{d u_2}{dt} = r_2 u_2 \left(1- \dfrac{u_2}{K_2} - b_{21}\dfrac{u_1}{K_1}\right)$$

Este é conhecido como o modelo Lotka-Volterra para competição de duas espécies. 


Começamos então, com o Sympy, declarando com quais variáveis e parâmetros estamos trabalhando. Fazemos isso a partir da função *symbols*:

In [ ]:
# Declaramos as variáveis u1 e u2 e as armazenamos em uma lista"Variaveis" 
u_1,u_2=symbols('u_1 u_2')
Variaveis=[u_1,u_2]

# Declaramos cada parâmetro e os armazenamos em uma lista "Parametros" 
r_1, r_2, K_1, K_2, b_12, b_21=symbols('r_1 r_2 K_1 K_2 b_12 b_21')
Parametros=[r_1, r_2, K_1, K_2, b_12, b_21]

Agora basta escrever as equações do modelo a partir dos símbolos que acabamos de definir. É conveniente guardas o lado direito das equações do modelo em uma lista "Modelos":

In [ ]:
# Guardamos o lado direito das equações do modelo na lista Modelo
# É muito importante preservar a ordem que construímos a lista Variaveis.
# Como guardamos primeiro u_1 e depois o u_2, a lista das equações do modelo 
# deve ter como a primeira entrada a derivada de u_1 em relação ao tempo
# e em seguida a derivada de u_2 em relação ao tempo.


#                   du_1/dt                                  du_2/dt
Modelo=[r_1*u_1*(1- u_1/K_1  - b_12*u_2/K_2), r_2*u_2*(1- u_2/K_2  - b_21*u_1/K_1)]

Pronto! o modelo está definido e agora podemos manipular as expressões com funções do *Sympy*

## Pontos Fixos

Um ponto fixo é uma configuração de um sistema em que não ha variação das variáveis de interesse. No contexto os modelos ecológicos, seriam pontos em que as derivadas dos tamanhos populacionáis são nulas, ou seja, não há variação das populações estudadas.

Para o modelo de competição que usamos como exemplo, um ponto fixo é qualquer ponto $u^* = \{u_1^*, u_2^*\}$ que resulte em:

$$\displaystyle \left.\dfrac{du_1}{dt}\right|_{u^*}= \left.\dfrac{du_2}{dt}\right|_{u^*} =0$$

Ou seja, as derivadas de $u_1$ e $u_2$ avaliadas em $u^*$ são nulas. Dessa forma, o lado esquerdo das equações do modelo serão iguais a zero quando o sistema se encontra em um ponto fixo:

$$\displaystyle r_1 u_1^* \left(1- \dfrac{u_1^*}{K_1} - b_{12}\dfrac{u_2^*}{K_2}\right)=0$$
$$\displaystyle  r_2 u_2^* \left(1- \dfrac{u_2^*}{K_2} - b_{21}\dfrac{u_1^*}{K_1}\right)=0$$

Encontrar os pontos fixos do sistema entã se torna um problema de aritimética. Basta encontrar os valores de $u_1^*$ e $u_2^*$ que satisfazem o sistema acima.

Uma das grandes vantagens de usar o *Sympy* é poder resolver este sistema, que muitas vezes pode ser bastante complicado, com apenas uma função: 

In [ ]:
# A função nonlinsolve iguala o lado direito das equações do modelo a zero  
# e nos retorna as expressões de u_1 e u_2 que satisfazem este sistema.
# Devemos colocar como input o lado direito das equações e as variáveis de
# interesse:
Pontos_Fixos=nonlinsolve(Modelo, Variaveis)

# As vezes o resultado pode ser sair bastante desorganizado.
# A função simplify nos ajuda organizando e simplificando 
# a expressão matemática:
Pontos_Fixos= simplify(Pontos_Fixos)

# Por fim, printamos o resultado:
Pontos_Fixos

Sucesso! Cada par do output, encapsulado por parênteses, representa um ponto fixo, de modo que o primeiro termo representa o valor de $u_1^*$ e o segundo o valor de $u_2^*$. Dessa forma, a ordem da lista "Variaveis" é preservada nos resultados da função. 

Vamos agora rapidamente analisar os resultados. Temos:

- Um ponto fixo em que o tamanho populacional das duas espécies é nulo: $u_1^*=u_2^*=0$
- Um ponto fixo onde a espécie 2 não sobrevive, $u_2^*=0$, e a espécie 1 se encontra em sua capacidade suporte $u_1^*=K_1$
- Um ponto fixo onde a espécie 1 não sobrevive, $u_1^*=0$, e a espécie 2 se encontra em sua capacidade suporte $u_2^*=K_2$
- Um ponto de coexistência de ambas as espécies

## Matriz Jacobiana

De modo a seguir a análise do modelo, devemos escrever a matriz jacobiana modelo. O *Sympy* nos permite realizar esse passo com uma única função!

In [ ]:
#Para calcular a matriz jacobiana do modelo, devemos transformar as listas
# Variaveis e Modelo em matrizes do Sympy.
M=Matrix(Modelo)
V=Matrix(Variaveis)

#Chamamos a função Jacobian que nos dá a matriz jacobiana do modelo M em função das variáveis V
J=M.jacobian(V)

#Simplificando temos:
simplify(J)

## Estabilidade de cada ponto fixo

De modo a determinar as condições de estabilidade de um ponto fixo, devemos avaliar a matriz jacobiana do sistema neste ponto. Os autovalores da matriz estabelecem as condições de estabilidade:

- Caso a parte real de todos os autovalores sejam negativos, este é um ponto fixo **estável**.
- Caso a parte real de algum autovalor for positivo, denominamos este um ponto fixo **instável**.
- A existência de algum autovalor com parte real nula torna este ponto fixo **Indeterminado**.

Para isso, basta utilizar a função "subs" que substitui as expressões das pontos fixos obtidas previamente na matriz jacobiana.

Vamos obter as condições de estabilidade de cada ponto fixo!

### Ponto Fixo 1 ($u_1=0$ e $u_2=0$)

In [ ]:
#Substituindo u_1=0 e u_2=0
J0=J.subs([(u_1,0), (u_2,0)])

#simplificado
simplify(J0.eigenvals())

O output pode ser lido da seguinte forma:

{Autovalor 1 : Multiplicidade do autovalor 1, Autovalor 2 : Multiplicidade do autovalor 2}

Não estamos interessados na multiplicidade dos autovalores, apenas com o seu sinal. Dessa forma, este ponto fixo é estável apenas se:

 $$r_1 <0$$ e $$r_2 <0$$
 
Ou seja, caso a taxa de crescimento de ambas as espécies sejam negativos, o ponto fixo que representa a não sobrevivência de ambas é estável.

### Ponto Fixo 2 ($u_1=0$ e $u_2=K_2$)

In [ ]:
#Substituindo u_1=0 e u_2=K_2
J1=J.subs([(u_1,0), (u_2,K_2)])

#simplificado
simplify(J1.eigenvals())

Dessa forma, este ponto fixo é estável apenas se:

 $$r_1(1-b_{12})<0$$ e $$r_2 >0$$
 


### Ponro fixo 3 ($u_1 =K_1$, $u_2 = K_2$)

In [ ]:
#Substituindo u_1=K_1 e u_2=0
J2=J.subs([(u_1,K_1), (u_2,0)])

#simplificado
simplify(J2.eigenvals())

Dessa forma, este ponto fixo é estável apenas se:

 $$r_2(1-b_{21})<0$$ e $$r_1 >0$$
 


### Ponto fixo de coexistência

In [ ]:
#Ponto de coexistência
J3=J.subs([(u_1,K_1*(b_12 - 1)/(b_12*b_21 - 1)), (u_2,(K_2*b_21 - K_2)/(b_12*b_21 - 1))])

#simplificando
simplify(J3.eigenvals())

Opa! Aqui percebemos uma das vantagens do *Sympy*. Uma expressão como essa é bastante complicada e realizar estas contas na mão seria muito trabalhoso.

Claro que não é muito prático trabalhar analiticamente com uma expressão desta, mas sempre podemos apenas introduzir os valores dos parâmetros na expressão e verificar as condições de estabilidade graficamente.

Também podemos utilizar outros métodos, como integração numérica do modelo para realizar este tipo de análise.